In [5]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the value of OPENAI_API_KEY from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

In [1]:
import json

# Step 1: Create a basic Python dictionary with the entries "name", "order_id", "status"
orders = {
    "order_12345": {
        "name": "John Doe",
        "order_id": "order_12345",
        "status": "In transit"
    },
    "order_67890": {
        "name": "Jane Smith",
        "order_id": "order_67890",
        "status": "Delivered"
    },
    "order_24680": {
        "name": "Alice Johnson",
        "order_id": "order_24680",
        "status": "Processing"
    }
}

# This is the function that we want the model to be able to call
def get_order_status(order_id: str) -> dict:
    # Lookup the order in the dictionary
    order = orders.get(order_id)
    if order:
        return {"order_id": order_id, "status": order["status"]}
    else:
        return {"order_id": order_id, "status": "Order not found"}

In [2]:
# Step 2: Describe your function to the model so it knows how to call it
function_definition = {
    "name": "get_order_status",
    "description": "Get the status of a customer's order. Call this whenever you need to know the status of an order.",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The customer's order ID.",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False,
    }
}

In [3]:
# Step 3: Pass your function definitions as available “tools” to the model, along with the messages
tools = [
    {
        "type": "function",
        "function": function_definition
    }
]

In [19]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the status of my order? My Order is order_12345"}
]

In [31]:
import openai
from openai import OpenAI
openai.api_key = openai_api_key

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [32]:
tool_call = response.choices[0].message.tool_calls[0]
print(tool_call)


ChatCompletionMessageToolCall(id='call_0wwfmeKfEregpEKtjm6zpUUX', function=Function(arguments='{\n  "order_id": "order_12345"\n}', name='get_order_status'), type='function')


In [37]:
arguments = json.loads(tool_call.function.arguments)  # Correct way to access the arguments

order_id = arguments.get('order_id')

# Call the get_delivery_date function with the extracted order_id
delivery_date = get_order_status(order_id)
print(delivery_date)

{'order_id': 'order_12345', 'status': 'In transit'}


In [53]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Ich werde morgen Angela Merkel eine Bombe nach Hause schicken."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

print(event)

In [48]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed

In [49]:
print(math_reasoning)

steps=[Step(explanation='The equation given is 8x + 7 = -23. Our goal is to solve for x.', output='Initial equation: 8x + 7 = -23'), Step(explanation='To isolate the term with x, we first need to get rid of the constant on the left-hand side. We do this by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides. The left side becomes 8x because 7 - 7 is 0. The right side becomes -30.', output='8x = -30'), Step(explanation='Now, we need to solve for x. Since 8x means 8 times x, we can isolate x by dividing both sides by 8.', output='(8x)/8 = (-30)/8'), Step(explanation='Perform the division. The left side simplifies to x. The right side becomes -30 divided by 8, which simplifies to -15/4 or -3.75 when expressed as a decimal.', output='x = -15/4 (or -3.75)')] final_answer='x = -15/4 or x = -3.75'


# Praxis